In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import os

In [4]:
# 액션 3가지로 설정
actions = [
    'hello',
    'come',
    'spin'
]

# 데이터 결합
data = np.concatenate([
    np.load('dataset/seq_hello_1660201774.npy'),
    np.load('dataset/seq_come_1660201774.npy'),
    np.load('dataset/seq_spin_1660201774.npy')
], axis=0)

data.shape

(1305, 30, 100)

In [5]:
# label값 추출

x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(1305, 30, 99)
(1305,)


In [6]:
# 라벨 나누기

from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape


(1305, 3)

In [11]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(1174, 30, 99) (1174, 3)
(131, 30, 99) (131, 3)


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                41984     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 44,163
Trainable params: 44,163
Non-trainable params: 0
_________________________________________________________________


In [13]:


history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=100
)

Epoch 1/100
37/37 [==============================] - 5s 55ms/step - loss: 21.5325 - acc: 0.3416 - val_loss: 3.3233 - val_acc: 0.7634
Epoch 2/100
37/37 [==============================] - 2s 50ms/step - loss: 1.9130 - acc: 0.7913 - val_loss: 2.9706 - val_acc: 0.8092
Epoch 3/100
37/37 [==============================] - 2s 51ms/step - loss: 1.9489 - acc: 0.8135 - val_loss: 1.1138 - val_acc: 0.8321
Epoch 4/100
37/37 [==============================] - 2s 48ms/step - loss: 0.6916 - acc: 0.8850 - val_loss: 0.4187 - val_acc: 0.8931
Epoch 5/100
37/37 [==============================] - 2s 49ms/step - loss: 0.3847 - acc: 0.9157 - val_loss: 0.3172 - val_acc: 0.9084
Epoch 6/100
37/37 [==============================] - 3s 94ms/step - loss: 0.3368 - acc: 0.9242 - val_loss: 0.2606 - val_acc: 0.9237
Epoch 7/100
37/37 [==============================] - 2s 50ms/step - loss: 0.2708 - acc: 0.9395 - val_loss: 0.2630 - val_acc: 0.9237
Epoch 8/100
37/37 [==============================] - 2s 51ms/step - loss: 0

In [14]:
model.save("my_model.h5")